# Mask Scorer

## Description

This script calculates performance scores that measure the accuracy of a system output mask to a reference mask. The script generates CSV report tables, one containing scores for each mask and another containing an average of the scores in the first CSV. Given the option, the script will generate a detailed HTML index file for the mask region performance results.

In the following metrics, the following terminology is used:
 * $GT$ refers to the ground truth mask
 * $sys$ refers to the system output mask
 * $TP$ refers to True Positives computed between the ground truth mask and the system output
 * $TN$ refers to True Negatives computed between the ground truth mask and the system output
 * $FN$ refers to False Negatives computed between the ground truth mask and the system output
 * $FP$ refers to False Positives computed between the ground truth mask and the system output
 * $weights$ is a matrix of 1's and 0's that denotes the set of pixels scored by the system; it is generated by a difference in the erosion and dilation of the manipulated area of $GT$.

The following metrics are used to score each mask:

### Nimble Mask Metric (NMM)
\begin{equation*}
NMM(GT,sys,weights,c)=\max{\left(\frac{TP - FN - FP}{\Sigma_{px\in GT}weights(px)},c\right)}
\end{equation*}

$\Sigma_{px \in GT}$ refers to the sum over the pixels in the ground truth that are marked black. $c$ denotes a minimum cutoff value for the scoring to have any meaning; by default, $c=-1$.

### Matthews Correlation Coefficient (MCC)
\begin{equation*}
MCC(GT,sys) = \frac{TP*TN - FP*FN}{\sqrt{(TP+FP)(TP+FN)(TN+FP)(TN+FN)}}
\end{equation*}

An MCC of 1 denotes perfect correlation, an MCC of 0 denotes no correlation at all, and an MCC of -1 denotes perfect anti-correlation.

### Weighted L1 Loss (BWL1 and GWL1)
\begin{equation*}
WL1(GT,sys,weights)=\frac{(FP+FN)_{weights > 0}}{\Sigma weights(px)}
\end{equation*}

A Weighted L1 of 0 denotes perfect or near perfect match up to variation within the weights that are 0; 1 denotes perfect mismatch. $(FP+FN)_{weights > 0}$ refers to the total number of $FP$ and $FN$ pixels where weights are greater than 0.

The Weighted L1 is applied separately to the original grayscale system output and the binarized mask, producing the grayscale Weighted L1 (GWL1) and binarized Weighted L1 (BWL1) metrics respectively. In the case of the original grayscale, the value is summed over the weighted difference in pixel intensity.

## Command-line Options

Example:
```
python2 MaskScorer.py -t manipulation --refDir ../../data/test_suite/maskScorerTests -r reference/manipulation/NC2017-manipulation-ref.csv -x indexes/NC2017-manipulation-index.csv -s ../../data/test_suite/maskScorerTests/B_NC2017_Manipulation_ImgOnly_c-me2_1/B_NC2017_Manipulation_ImgOnly_c-me2_1.csv -oR ../../data/test_suite/maskScorerTests/manipconfmanmade -html -q "ConfidenceScore < 0.5" "ManMade=='no'"
```
Running this code would produce, under the maskoutputs directory, an aggregate report of the computed mask scores titled sample.csv and a per-image score report titled sample-perimage.csv for the removal task. The -html flag is also set, allowing the code to generate an HTML per-image index file with the scores and metadata containing links to individual detailed reports of each image (described in the Sample HTML Output at the bottom of the page).

The command-line options for the mask scorer can be categorized as follows:

### Task Type Options

-t --task [manipulation, splice]

  * Specify the task type for evaluation (default = manipulation)

### Input Options

All CSV files passed to the Mask Scorer must contain headers and must have their rows separated by pipe characters ('|'). Fields and values in the CSV should <i>not</i> be enclosed in quotes ( ' or " ) if possible (e.g. entries 'foo', an empty field, and 'bar', in that order, should look like this in the csv: foo||bar). Additional specifications for the index and system output files can be found in the ValidatorNotebook.html file under the Validator directory.

--refDir

  * Specify the reference and index data path (e.g. "/NC2016_Test0601") (default = .)


-r --inRef

  * Specify the reference CSV file within refDir that contains the ground-truth information and metadata about each image. Key fields are TaskID, ProbeFileID, ProbeFileName, and ProbeMaskFileName, and if scoring on the 'splice' task, DonorFileID, DonorFileName, and DonorMaskFileName as well. Often the File ID's for the Probe and Donor will be the same as the file names, minus the extension. Additional fields, especially metadata pertaining to the ground-truth manipulation, may be included.

-x --inIndex

  * Define the index CSV file within refDir. The index file contains the TaskID, ProbeFileID, ProbeFileName, ProbeWidth, and ProbeHeight fields, and if scoring on the splice task, the DonorFileID, DonorFileName, DonorWidth, and DonorHeight fields as well. No additional fields are permitted for the index file.

--sysDir

  * Specify the system output data path, for example "mysysoutput/" (default = .) 


-s --inSys

  * Specify the CSV file of the system performance results formatted according to NC2016 specification. The file must contain the ProbeFileID, ConfidenceScore, and ProbeOutputMaskFileName fields, in that order, and if scoring on the splice task, the ProbeFileID, DonorFileID, ConfidenceScore, ProbeOutputMaskFileName, and DonorOutputMaskFileName fields, in that order. The ProbeOutputMaskFileNames and DonorOutputMaskFileNames (where relevant) should be directory strings relative to the location of the system performance CSV.

--rbin

  * Binarize the reference mask to black and white with a numeric threshold in the interval [0,255]. Choose -1 to not binarize and leave the mask as is. (default = -1)

--sbin

  * Binarize the system output mask to black and white with a numeric threshold in the interval [0,255]. Pick -1 to binarize by the threshold that gives the maximum absolute MCC. (default = -1)

-tmt --targetManiType

  * An array of manipulation tasks to be scored, separated by commas, e.g. 'Removal,Splice'. The default 'all' means that the system output will be scored against every manipulated region of the reference masks. (default: 'all')

### Output Options

--outRoot

  * Specify the report output path and the file name suffix for saving the plot and table (e.g., test/sys_xxx). For example, if you specify "--outRoot test/NIST_001", you will find the aggregate score report "NIST_001.csv" and the per-image report "NIST_001-perimage.csv" in the "test" folder.


### Scoring Options

--eks

  * Erosion kernel size. (number must be odd; default = 15)
  
--dks

  * Dilation kernel size. (number must be odd; default = 9)
  
-k kernel

  * The shape of the kernel to be used, for both erosion and dilation. Choose from 'box','disc','diamond','gaussian', or 'line'. The default kernel is 'box'.

### Performance Evaluation by Query

This option allows the user to evaluate their algorithm performance on either subsets or partitions of the data based on the specified queries and query options. The reference and index CSV files contain a list of factors (e.g., ProbePostProcessed|DonorPostProcessed|ManipulationQuality|IsManipulationTypeRemoval|...). Selecting neither of the following factors will output a single report table (CSV) over the entire computed dataset.

-q query
 * Evaluate algorithm performance on a partitioned dataset using multiple factor queries, one at a time (e.g. "Collection==['NC2017'] & Purpose==['add','remove']" will average over the rows that fit this criterion for one queried average, but "Collection==['NC2017']" "Purpose==['add','remove']" will average over the first and then the second independently for two queried averages). The option generates N report tables (CSV), one for each query.
   * Syntax: -q "query1" "query2" "query3" ...

-qp queryPartition
 * Uses one factor query to evaluate algorithm performance on a partitioned dataset through its individual sub-queries (e.g. "Collection==['NC2017'] & Purpose==['add','remove']" will average over "Collection==['NC2017'] & Purpose==['add']" and "Collection==['NC2017'] & Purpose==['remove']" for a total of two queried averages). This option generates a single report table (CSV) that contains M partition results, one result for each query.
   * Syntax: -qp "query"
   
-qm queryManipulation
 * Filters the dataset before scoring takes place for some number of queries. It is functionally similar to the -q query option. The option generates M report tables (CSV), one for each query.
    * Syntax: -qm "query1" "query2" "query3" ...

### Report Options

-v verbose

  * Control print output. Select 1 to print all non-error related output and 0 to suppress all print output (bar argument-parsing errors).
  
--precision

  * The number of digits to round computed scores, (e.g. a score of 0.3333333333333... will round to 0.33333 for a precision of 5), (default = 16).

-html

  * Output the report to HTML files. Set the flag to choose this option.

## Sample HTML Output

The following set of images is a sample of what you should see in the HTML report for each image. You can access each image from the appropriate link in the index.html file that is produced in your chosen output directory. (The sample output below has its links removed for the sake of display and is formatted according to what a hyperlink on your system might look like.) Following is a sample index file (scroll left if the table is truncated by the display to view the rest of the table):

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: center;">
      <th></th>
      <th>TaskID</th>
      <th>ProbeFileID</th>
      <th>ProbeFileName</th>
      <th>ProbeMaskFileName</th>
      <th>IsTarget</th>
      <th>OutputProbeMaskFileName</th>
      <th>ConfidenceScore</th>
      <th>NMM</th>
      <th>MCC</th>
      <th>WL1</th>
      <th>Scored</th>
      <th>TargetManipulations</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>manipulation</td>
      <td>43e1d6f0a9306a629a51062729549d76</td>
      <td><u><font style="color:#0000FF">43e1d6f0a9306a629a51062729549d76.jpg</font></u></td>
      <td>reference/manipulation/mask/77232d3f2f9e866ad7bbc5faccd25119.png</td>
      <td>Y</td>
      <td>mask/leaves-mask.png</td>
      <td>0.70</td>
      <td>0.109</td>
      <td>0.650</td>
      <td>0.092</td>
      <td>Y</td>
      <td>all</td>
    </tr>
    <tr>
      <th>1</th>
      <td>manipulation</td>
      <td>07e65ca7be451f9fc756e9cc7a458eb6</td>
      <td><u><font style="color:#0000FF">07e65ca7be451f9fc756e9cc7a458eb6.jpg</font></u></td>
      <td>reference/manipulation/mask/f50af0d7f4480386c12494f1146cdf39.png</td>
      <td>Y</td>
      <td>mask/plus-mask.png</td>
      <td>0.30</td>
      <td>-1.000</td>
      <td>0.418</td>
      <td>0.061</td>
      <td>Y</td>
      <td>all</td>
    </tr>
    <tr>
      <th>2</th>
      <td>manipulation</td>
      <td>43e1d6f0a9306a629a51062729549d76-2</td>
      <td><u><font style="color:#0000FF">43e1d6f0a9306a629a51062729549d76-2.jpg</font></u></td>
      <td>reference/manipulation/mask/77232d3f2f9e866ad7bbc5faccd25119-2.png</td>
      <td>Y</td>
      <td>mask/leaves-mask-graydient.png</td>
      <td>0.71</td>
      <td>0.116</td>
      <td>0.653</td>
      <td>0.091</td>
      <td>Y</td>
      <td>all</td>
    </tr>
    <tr>
      <th>3</th>
      <td>manipulation</td>
      <td>43e1d6f0a9306a629a51062729549d76-3</td>
      <td><u><font style="color:#0000FF">43e1d6f0a9306a629a51062729549d76-3.jpg</font></u></td>
      <td>reference/manipulation/mask/77232d3f2f9e866ad7bbc5faccd25119-3.png</td>
      <td>Y</td>
      <td>mask/leaves-mask.png</td>
      <td>0.70</td>
      <td>0.109</td>
      <td>0.650</td>
      <td>0.092</td>
      <td>Y</td>
      <td>all</td>
    </tr>
  </tbody>
</table>
<h3>Average Scores</h3>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: center;">
      <th></th>
      <th>TaskID</th>
      <th>Query</th>
      <th>TargetManipulations</th>
      <th>NMM</th>
      <th>MCC</th>
      <th>WL1</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>manipulation</td>
      <td>ConfidenceScore &lt; 0.5</td>
      <td>all</td>
      <td>-1.000</td>
      <td>0.418</td>
      <td>0.061</td>
    </tr>
    <tr>
      <th>1</th>
      <td>manipulation</td>
      <td>ManMade=='no'</td>
      <td>all</td>
      <td>0.111</td>
      <td>0.651</td>
      <td>0.092</td>
    </tr>
  </tbody>
</table>

Some columns in this table containing the metadata of the files have been truncated for the sake of display. Multiple rows may be displayed in the Average Scores table depending on the query option and query(ies) being passed; the one shown has one for ConfidenceScores less than 0.5 and another for non-manmade images.

Reports for each image are located in hyperlinks (formatted in the above example for display) under ProbeFileName for images that can be evaluated. Probe and donor masks for each ProbeFileID in the splice task will have one report each.

A sample page for one of the links is shown below:


<h2>Manipulated Image: 07e65ca7be451f9fc756e9cc7a458eb6.jpg </h2><br/>
<img src="notebookImgs/07e65ca7be451f9fc756e9cc7a458eb6.jpg" alt="manipulated image" style="width:640px;">
<br/>
<table>
  <tbody>
    <tr>
      <td>
<h3>Composite with Color Mask: plus-mask_composite.jpg </h3>
<img src="notebookImgs/plus-mask_composite.jpg" alt="manip image plus mask" style="width:640px;">
      </td>
      <td width="480px" align="center">
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Purpose</th>
      <th>Color</th>
      <th>Evaluated</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>remove</td>
      <td bgcolor="#FF0000"></td>
      <td>Y</td>
    </tr>
    <tr>
      <td>add</td>
      <td bgcolor="#00FF00"></td>
      <td>Y</td>
    </tr>
  </tbody>
</table>
      </td>
    </tr>
  </tbody>
</table>
<br/>
<table border="1">
  <tbody>
    <tr>
      <td><img src="notebookImgs/f50af0d7f4480386c12494f1146cdf39.png" alt="reference mask" style="width:640px;"><br/><b>Colorized Reference Mask</b></td>
      <td><img src="notebookImgs/plus-mask.png" alt="system output mask" style="width:640px;"><br/><b>Original System Output Mask</b></td>
    </tr>
    <tr>
      <td><img src="notebookImgs/f50af0d7f4480386c12494f1146cdf39-bin.png" alt="binarized reference mask" style="width:640px;"><br/><b>Binarized Reference Mask (Black = Manipulated, Yellow = Boundary No-Score Zone, Pink = Selective No-Score Zone)</b></td>
      <td><img src="notebookImgs/plus-mask-bin.png" alt="binarized system output mask" style="width:640px;"><br/><b>Binarized System Output Mask (Black = Manipulated, Threshold &lt;= 0)</b></td>
    </tr>
    <tr>
      <td><img src="notebookImgs/plus-mask-weights.png" alt="no-score zone" style="width:640px;"><br/><b>No-Score Zone (Yellow = Boundary No-Score Zone, Pink = Selective No-Score Zone)</b></td>
      <td><img src="notebookImgs/plus-mask_colored.jpg" alt="color mask" style="width:640px;"><br/><b>Evaluation Result Visualization</b></td>
    </tr>
    <tr>
      <td><b>NIMBLE Mask Metric (NMM)</b>: -1.0 <br/>
      <b>Matthews Correlation Coefficient (MCC)</b>: 0.418 <br/>
      <b>Binary Weighted L1 Loss (WL1)</b>: 0.061 <br/>
      <b>Grayscale Weighted L1 Loss (WL1)</b>: 0.061 <br/></td>
      <td><b>Total Pixels</b>: 14096788 <br/>
<table style="border=1; background-color:#C8C8C8">
  <tbody>
    <tr>
      <th>
      Confusion Measures
      </th>
      <th>
      Pixels
      </th>      
      <th>
      Proportion
      </th>      
    </tr>
    <tr>
      <td>
<b>True Positives (<font style="color:#00CF00">TP: green</font>)</b>:
      </td>
      <td align="left">
      276301
      </td>
      <td align="left">
      0.02
      </td>
    </tr>
    <tr>
      <td>
<b>False Positives (<font style="color:#FF0000">FP: red</font>)</b>:
      </td>
      <td align="left">
      754696
      </td>
      <td align="left">
      0.054
      </td>
    </tr>
    <tr>
      <td>
<b>True Negatives (<font style="color:#FFFFFF">TN: white</font>)</b>:
      </td>
      <td align="left">
      12962609
      </td>
      <td align="left">
      0.92
      </td>
    </tr>
    <tr>
      <td>
<b>False Negatives (<font style="color:#3333FF">FN: blue</font>)</b>:
      </td>
      <td align="left">
      103182
      </td>
      <td align="left">
      0.007
      </td>
    </tr>
    <tr>
      <td>
<b>Boundary No-Score Zone (<font style="color:#FFFF00">BNS: yellow</font>)</b>:
      </td>
      <td align="left">
      208428
      </td>
      <td align="left">
      0.015
      </td>
    </tr>
    <tr>
      <td>
<b>Selective No-Score Zone (<font style="color:#FFB6C1">SNS: pink</font>)</b>:
      </td>
      <td align="left">
      0
      </td>
      <td align="left">
      0.0
      </td>
    </tr>
  </tbody>
</table>
</td>
    </tr>
  </tbody>
</table>
<br/>


The user may also select which manipulation regions to score, depending on the manipulations listed under the "Purpose" column in the journalMask file. The page below is a sample page of a manipulation with selected regions (based on manipulation type) being scored. Other regions are dilated by a separate factor and counted as selective no-score zones (pink) in addition to the boundary no-score zones applied to the regions of interest (yellow).

Also demonstrated is the thresholding of the mask by the threshold that yields the maximum MCC. The resulting table of thresholds computed is included in at the bottom of the page.

<h2>Manipulated Image: 43e1d6f0a9306a629a51062729549d76-2.jpg </h2><br/>
<img src="notebookImgs/43e1d6f0a9306a629a51062729549d76-2.jpg" alt="manipulated image" style="width:640px;">
<br/>
<table>
  <tbody>
    <tr>
      <td>
<h3>Composite with Color Mask: leaves-mask-graydient_composite.jpg </h3>
<img src="notebookImgs/leaves-mask-graydient_composite.jpg" alt="manip image plus mask" style="width:640px;">
      </td>
      <td width="480px" align="center">
<h3>Target Manipulations: add</h3>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Purpose</th>
      <th>Color</th>
      <th>Evaluated</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>add</td>
      <td bgcolor="#00F7FA"></td>
      <td>Y</td>
    </tr>
    <tr>
      <td>remove</td>
      <td bgcolor="#FF00F0"></td>
      <td>N</td>
    </tr>
    <tr>
      <td>add</td>
      <td bgcolor="#00FF06"></td>
      <td>Y</td>
    </tr>
    <tr>
      <td>add</td>
      <td bgcolor="#F4FD00"></td>
      <td>Y</td>
    </tr>
    <tr>
      <td>clone</td>
      <td bgcolor="#FF0C00"></td>
      <td>N</td>
    </tr>
    <tr>
      <td>heal</td>
      <td bgcolor="#FFFF7F"></td>
      <td>N</td>
    </tr>
  </tbody>
</table>
      </td>
    </tr>
  </tbody>
</table>
<br/>
<table border="1">
  <tbody>
    <tr>
      <td><img src="notebookImgs/77232d3f2f9e866ad7bbc5faccd25119-2.png" alt="reference mask" style="width:640px;"><br/><b>Colorized Reference Mask</b></td>
      <td><img src="notebookImgs/leaves-mask-graydient.png" alt="system output mask" style="width:640px;"><br/><b>Original System Output Mask</b></td>
    </tr>
    <tr>
      <td><img src="notebookImgs/77232d3f2f9e866ad7bbc5faccd25119-2-bin.png" alt="binarized reference mask" style="width:640px;"><br/><b>Binarized Reference Mask (Black = Manipulated, Yellow = Boundary No-Score Zone, Pink = Selective No-Score Zone)</b></td>
      <td><img src="notebookImgs/leaves-mask-graydient-bin.png" alt="binarized system output mask" style="width:640px;"><br/><b>Binarized System Output Mask (Black = Manipulated, Threshold &lt;= 128)</b></td>
    </tr>
    <tr>
      <td><img src="notebookImgs/leaves-mask-graydient-weights.png" alt="no-score zone" style="width:640px;"><br/><b>No-Score Zone (Yellow = Boundary No-Score Zone, Pink = Selective No-Score Zone)</b></td>
      <td><img src="notebookImgs/leaves-mask-graydient_colored.jpg" alt="color mask" style="width:640px;"><br/><b>Evaluation Result Visualization</b></td>
    </tr>
    <tr>
      <td><b>NIMBLE Mask Metric (NMM)</b>: 0.293 <br/>
      <b>Matthews Correlation Coefficient (MCC)</b>: 0.721 <br/>
      <b>Weighted L1 Loss (WL1)</b>: 0.074 <br/></td>
      <td><b>Total Pixels</b>: 9227609 <br/>
<table style="border:1;background:#C8C8C8">
  <tbody>
    <tr>
      <th>
      Confusion Measures
      </th>
      <th>
      Pixels
      </th>      
      <th>
      Proportion
      </th>      
    </tr>
    <tr>
      <td>
<b>True Positives (<font style="color:#00CF00">TP: green</font>)</b>:
      </td>
      <td>
      1012095
      </td>
      <td>
      0.11
      </td>
    </tr>
    <tr>
      <td>
<b>False Positives (<font style="color:#FF0000">FP: red</font>)</b>:
      </td>
      <td>
      588762
      </td>
      <td>
      0.064
      </td>
    </tr>
    <tr>
      <td>
<b>True Negatives (<font style="color:#FFFFFF">TN: white</font>)</b>:
      </td>
      <td>
      7528296
      </td>
      <td>
      0.816
      </td>
    </tr>
    <tr>
      <td>
<b>False Negatives (<font style="color:#3333FF">FN: blue</font>)</b>:
      </td>
      <td>
      98456
      </td>
      <td>
      0.011
      </td>
    </tr>
    <tr>
      <td>
<b>Boundary No-Score Zone (<font style="color:#FFFF00">BNS: yellow</font>)</b>:
      </td>
      <td>
      160636
      </td>
      <td>
      0.017
      </td>
    </tr>
    <tr>
      <td>
<b>Selective No-Score Zone (<font style="color:#FFB6C1">SNS: pink</font>)</b>:
      </td>
      <td>
      348211
      </td>
      <td>
      0.038
      </td>
    </tr>
  </tbody>
</table>
</td>
    </tr>
  </tbody>
</table>
<br/>
<h4>Measures for Each Threshold</h4><br/><table border="1" class="dataframe">
  <thead>
    <tr style="text-align: center;">
      <th>Threshold</th>
      <th>NMM</th>
      <th>MCC</th>
      <th>WL1</th>
      <th>TP</th>
      <th>TN</th>
      <th>FP</th>
      <th>FN</th>
      <th>BNS</th>
      <th>SNS</th>
      <th>N</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>0</td>
      <td>-0.969815</td>
      <td>0.114790</td>
      <td>0.118546</td>
      <td>16864</td>
      <td>8116852</td>
      <td>206</td>
      <td>1093687</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>1</td>
      <td>-0.937568</td>
      <td>0.165630</td>
      <td>0.116606</td>
      <td>34774</td>
      <td>8116844</td>
      <td>214</td>
      <td>1075777</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>2</td>
      <td>-0.904727</td>
      <td>0.204977</td>
      <td>0.114629</td>
      <td>53010</td>
      <td>8116844</td>
      <td>214</td>
      <td>1057541</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>3</td>
      <td>-0.871658</td>
      <td>0.238239</td>
      <td>0.112639</td>
      <td>71372</td>
      <td>8116844</td>
      <td>214</td>
      <td>1039179</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>4</td>
      <td>-0.838644</td>
      <td>0.267460</td>
      <td>0.110653</td>
      <td>89704</td>
      <td>8116844</td>
      <td>214</td>
      <td>1020847</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>5</td>
      <td>-0.805765</td>
      <td>0.293786</td>
      <td>0.108674</td>
      <td>107961</td>
      <td>8116844</td>
      <td>214</td>
      <td>1002590</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>6</td>
      <td>-0.775710</td>
      <td>0.316021</td>
      <td>0.106866</td>
      <td>124650</td>
      <td>8116844</td>
      <td>214</td>
      <td>985901</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>7</td>
      <td>-0.747532</td>
      <td>0.335044</td>
      <td>0.105204</td>
      <td>140608</td>
      <td>8116221</td>
      <td>837</td>
      <td>969943</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>8</td>
      <td>-0.720874</td>
      <td>0.351076</td>
      <td>0.103703</td>
      <td>156368</td>
      <td>8114306</td>
      <td>2752</td>
      <td>954183</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>9</td>
      <td>-0.694722</td>
      <td>0.366052</td>
      <td>0.102244</td>
      <td>171944</td>
      <td>8112197</td>
      <td>4861</td>
      <td>938607</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>10</td>
      <td>-0.668920</td>
      <td>0.380207</td>
      <td>0.100815</td>
      <td>187411</td>
      <td>8109917</td>
      <td>7141</td>
      <td>923140</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>11</td>
      <td>-0.643377</td>
      <td>0.393726</td>
      <td>0.099407</td>
      <td>202785</td>
      <td>8107536</td>
      <td>9522</td>
      <td>907766</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>12</td>
      <td>-0.618041</td>
      <td>0.406721</td>
      <td>0.098015</td>
      <td>218077</td>
      <td>8105089</td>
      <td>11969</td>
      <td>892474</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>13</td>
      <td>-0.592952</td>
      <td>0.419224</td>
      <td>0.096640</td>
      <td>233256</td>
      <td>8102594</td>
      <td>14464</td>
      <td>877295</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>14</td>
      <td>-0.568173</td>
      <td>0.431221</td>
      <td>0.095289</td>
      <td>248305</td>
      <td>8100014</td>
      <td>17044</td>
      <td>862246</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>15</td>
      <td>-0.543681</td>
      <td>0.442785</td>
      <td>0.093957</td>
      <td>263217</td>
      <td>8097390</td>
      <td>19668</td>
      <td>847334</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>16</td>
      <td>-0.519471</td>
      <td>0.453907</td>
      <td>0.092650</td>
      <td>278035</td>
      <td>8094640</td>
      <td>22418</td>
      <td>832516</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>17</td>
      <td>-0.495532</td>
      <td>0.464634</td>
      <td>0.091364</td>
      <td>292757</td>
      <td>8091781</td>
      <td>25277</td>
      <td>817794</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>18</td>
      <td>-0.471820</td>
      <td>0.475033</td>
      <td>0.090096</td>
      <td>307388</td>
      <td>8088853</td>
      <td>28205</td>
      <td>803163</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>19</td>
      <td>-0.448370</td>
      <td>0.485106</td>
      <td>0.088847</td>
      <td>321907</td>
      <td>8085857</td>
      <td>31201</td>
      <td>788644</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>20</td>
      <td>-0.425092</td>
      <td>0.494961</td>
      <td>0.087606</td>
      <td>336312</td>
      <td>8082899</td>
      <td>34159</td>
      <td>774239</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>21</td>
      <td>-0.402127</td>
      <td>0.504467</td>
      <td>0.086393</td>
      <td>350616</td>
      <td>8079795</td>
      <td>37263</td>
      <td>759935</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>22</td>
      <td>-0.379497</td>
      <td>0.513635</td>
      <td>0.085207</td>
      <td>364806</td>
      <td>8076546</td>
      <td>40512</td>
      <td>745745</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>23</td>
      <td>-0.357146</td>
      <td>0.522533</td>
      <td>0.084042</td>
      <td>378881</td>
      <td>8073218</td>
      <td>43840</td>
      <td>731670</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>24</td>
      <td>-0.335045</td>
      <td>0.531185</td>
      <td>0.082898</td>
      <td>392861</td>
      <td>8069802</td>
      <td>47256</td>
      <td>717690</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>25</td>
      <td>-0.313229</td>
      <td>0.539588</td>
      <td>0.081774</td>
      <td>406725</td>
      <td>8066302</td>
      <td>50756</td>
      <td>703826</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>26</td>
      <td>-0.291721</td>
      <td>0.547741</td>
      <td>0.080675</td>
      <td>420464</td>
      <td>8062710</td>
      <td>54348</td>
      <td>690087</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>27</td>
      <td>-0.270502</td>
      <td>0.555662</td>
      <td>0.079598</td>
      <td>434091</td>
      <td>8059021</td>
      <td>58037</td>
      <td>676460</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>28</td>
      <td>-0.249535</td>
      <td>0.563376</td>
      <td>0.078541</td>
      <td>447625</td>
      <td>8055238</td>
      <td>61820</td>
      <td>662926</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>29</td>
      <td>-0.228855</td>
      <td>0.570885</td>
      <td>0.077506</td>
      <td>461036</td>
      <td>8051382</td>
      <td>65676</td>
      <td>649515</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>30</td>
      <td>-0.209437</td>
      <td>0.577789</td>
      <td>0.076550</td>
      <td>473786</td>
      <td>8047447</td>
      <td>69611</td>
      <td>636765</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>31</td>
      <td>-0.191645</td>
      <td>0.583932</td>
      <td>0.075702</td>
      <td>485720</td>
      <td>8043338</td>
      <td>73720</td>
      <td>624831</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>32</td>
      <td>-0.175931</td>
      <td>0.589004</td>
      <td>0.075018</td>
      <td>496855</td>
      <td>8038519</td>
      <td>78539</td>
      <td>613696</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>33</td>
      <td>-0.162086</td>
      <td>0.593197</td>
      <td>0.074470</td>
      <td>507174</td>
      <td>8033256</td>
      <td>83802</td>
      <td>603377</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>34</td>
      <td>-0.149116</td>
      <td>0.596838</td>
      <td>0.074020</td>
      <td>517429</td>
      <td>8027150</td>
      <td>89908</td>
      <td>593122</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>35</td>
      <td>-0.136391</td>
      <td>0.600317</td>
      <td>0.073606</td>
      <td>527742</td>
      <td>8020656</td>
      <td>96402</td>
      <td>582809</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>36</td>
      <td>-0.123206</td>
      <td>0.604094</td>
      <td>0.073147</td>
      <td>538146</td>
      <td>8014490</td>
      <td>102568</td>
      <td>572405</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>37</td>
      <td>-0.109820</td>
      <td>0.608007</td>
      <td>0.072669</td>
      <td>548597</td>
      <td>8008454</td>
      <td>108604</td>
      <td>561954</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>38</td>
      <td>-0.096448</td>
      <td>0.611918</td>
      <td>0.072195</td>
      <td>559076</td>
      <td>8002347</td>
      <td>114711</td>
      <td>551475</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>39</td>
      <td>-0.083822</td>
      <td>0.615531</td>
      <td>0.071773</td>
      <td>569202</td>
      <td>7996116</td>
      <td>120942</td>
      <td>541349</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>40</td>
      <td>-0.071453</td>
      <td>0.619020</td>
      <td>0.071378</td>
      <td>579300</td>
      <td>7989657</td>
      <td>127401</td>
      <td>531251</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>41</td>
      <td>-0.059065</td>
      <td>0.622532</td>
      <td>0.070985</td>
      <td>589430</td>
      <td>7983154</td>
      <td>133904</td>
      <td>521121</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>42</td>
      <td>-0.046165</td>
      <td>0.626358</td>
      <td>0.070534</td>
      <td>599593</td>
      <td>7977154</td>
      <td>139904</td>
      <td>510958</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>43</td>
      <td>-0.032540</td>
      <td>0.630546</td>
      <td>0.070018</td>
      <td>609960</td>
      <td>7971552</td>
      <td>145506</td>
      <td>500591</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>44</td>
      <td>-0.018072</td>
      <td>0.635075</td>
      <td>0.069445</td>
      <td>620741</td>
      <td>7966057</td>
      <td>151001</td>
      <td>489810</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>45</td>
      <td>-0.003347</td>
      <td>0.639711</td>
      <td>0.068853</td>
      <td>631631</td>
      <td>7960630</td>
      <td>156428</td>
      <td>478920</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>46</td>
      <td>0.011154</td>
      <td>0.644264</td>
      <td>0.068272</td>
      <td>642377</td>
      <td>7955242</td>
      <td>161816</td>
      <td>468174</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>47</td>
      <td>0.025325</td>
      <td>0.648689</td>
      <td>0.067712</td>
      <td>652943</td>
      <td>7949848</td>
      <td>167210</td>
      <td>457608</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>48</td>
      <td>0.039102</td>
      <td>0.652948</td>
      <td>0.067180</td>
      <td>663336</td>
      <td>7944362</td>
      <td>172696</td>
      <td>447215</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>49</td>
      <td>0.052117</td>
      <td>0.656884</td>
      <td>0.066709</td>
      <td>673441</td>
      <td>7938606</td>
      <td>178452</td>
      <td>437110</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>50</td>
      <td>0.064830</td>
      <td>0.660672</td>
      <td>0.066270</td>
      <td>683511</td>
      <td>7932584</td>
      <td>184474</td>
      <td>427040</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>51</td>
      <td>0.077523</td>
      <td>0.664463</td>
      <td>0.065831</td>
      <td>693557</td>
      <td>7926588</td>
      <td>190470</td>
      <td>416994</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>52</td>
      <td>0.090530</td>
      <td>0.668414</td>
      <td>0.065357</td>
      <td>703631</td>
      <td>7920885</td>
      <td>196173</td>
      <td>406920</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>53</td>
      <td>0.103519</td>
      <td>0.672349</td>
      <td>0.064889</td>
      <td>713737</td>
      <td>7915098</td>
      <td>201960</td>
      <td>396814</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>54</td>
      <td>0.115891</td>
      <td>0.676031</td>
      <td>0.064471</td>
      <td>723619</td>
      <td>7909074</td>
      <td>207984</td>
      <td>386932</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>55</td>
      <td>0.128063</td>
      <td>0.679636</td>
      <td>0.064070</td>
      <td>733433</td>
      <td>7902963</td>
      <td>214095</td>
      <td>377118</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>56</td>
      <td>0.140003</td>
      <td>0.683147</td>
      <td>0.063690</td>
      <td>743189</td>
      <td>7896712</td>
      <td>220346</td>
      <td>367362</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>57</td>
      <td>0.151486</td>
      <td>0.686483</td>
      <td>0.063347</td>
      <td>752772</td>
      <td>7890298</td>
      <td>226760</td>
      <td>357779</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>58</td>
      <td>0.162703</td>
      <td>0.689715</td>
      <td>0.063028</td>
      <td>762287</td>
      <td>7883725</td>
      <td>233333</td>
      <td>348264</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>59</td>
      <td>0.173364</td>
      <td>0.692749</td>
      <td>0.062749</td>
      <td>771549</td>
      <td>7877041</td>
      <td>240017</td>
      <td>339002</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>60</td>
      <td>0.183254</td>
      <td>0.695520</td>
      <td>0.062517</td>
      <td>780395</td>
      <td>7870332</td>
      <td>246726</td>
      <td>330156</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>61</td>
      <td>0.192515</td>
      <td>0.698056</td>
      <td>0.062338</td>
      <td>789033</td>
      <td>7863341</td>
      <td>253717</td>
      <td>321518</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>62</td>
      <td>0.201407</td>
      <td>0.700442</td>
      <td>0.062202</td>
      <td>797653</td>
      <td>7855976</td>
      <td>261082</td>
      <td>312898</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>63</td>
      <td>0.210034</td>
      <td>0.702735</td>
      <td>0.062094</td>
      <td>806229</td>
      <td>7848405</td>
      <td>268653</td>
      <td>304322</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>64</td>
      <td>0.218307</td>
      <td>0.704920</td>
      <td>0.062009</td>
      <td>814637</td>
      <td>7840776</td>
      <td>276282</td>
      <td>295914</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>65</td>
      <td>0.226109</td>
      <td>0.706955</td>
      <td>0.061958</td>
      <td>822832</td>
      <td>7833051</td>
      <td>284007</td>
      <td>287719</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>66</td>
      <td>0.233253</td>
      <td>0.708765</td>
      <td>0.061958</td>
      <td>830765</td>
      <td>7825118</td>
      <td>291940</td>
      <td>279786</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>67</td>
      <td>0.239864</td>
      <td>0.710400</td>
      <td>0.062001</td>
      <td>838499</td>
      <td>7816992</td>
      <td>300066</td>
      <td>272052</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>68</td>
      <td>0.245960</td>
      <td>0.711867</td>
      <td>0.062086</td>
      <td>846058</td>
      <td>7808644</td>
      <td>308414</td>
      <td>264493</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>69</td>
      <td>0.251209</td>
      <td>0.713057</td>
      <td>0.062231</td>
      <td>853220</td>
      <td>7800149</td>
      <td>316909</td>
      <td>257331</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>70</td>
      <td>0.255296</td>
      <td>0.713887</td>
      <td>0.062436</td>
      <td>859658</td>
      <td>7791812</td>
      <td>325246</td>
      <td>250893</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>71</td>
      <td>0.259300</td>
      <td>0.714772</td>
      <td>0.062600</td>
      <td>865618</td>
      <td>7784339</td>
      <td>332719</td>
      <td>244933</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>72</td>
      <td>0.262481</td>
      <td>0.715422</td>
      <td>0.062790</td>
      <td>870900</td>
      <td>7777308</td>
      <td>339750</td>
      <td>239651</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>73</td>
      <td>0.265140</td>
      <td>0.715913</td>
      <td>0.063012</td>
      <td>875898</td>
      <td>7770265</td>
      <td>346793</td>
      <td>234653</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>74</td>
      <td>0.267338</td>
      <td>0.716257</td>
      <td>0.063267</td>
      <td>880700</td>
      <td>7763102</td>
      <td>353956</td>
      <td>229851</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>75</td>
      <td>0.269037</td>
      <td>0.716443</td>
      <td>0.063562</td>
      <td>885307</td>
      <td>7755774</td>
      <td>361284</td>
      <td>225244</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>76</td>
      <td>0.270275</td>
      <td>0.716483</td>
      <td>0.063894</td>
      <td>889747</td>
      <td>7748269</td>
      <td>368789</td>
      <td>220804</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>77</td>
      <td>0.271154</td>
      <td>0.716415</td>
      <td>0.064256</td>
      <td>894055</td>
      <td>7740629</td>
      <td>376429</td>
      <td>216496</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>78</td>
      <td>0.271769</td>
      <td>0.716276</td>
      <td>0.064637</td>
      <td>898256</td>
      <td>7732910</td>
      <td>384148</td>
      <td>212295</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>79</td>
      <td>0.272227</td>
      <td>0.716104</td>
      <td>0.065029</td>
      <td>902385</td>
      <td>7725161</td>
      <td>391897</td>
      <td>208166</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>80</td>
      <td>0.272539</td>
      <td>0.715905</td>
      <td>0.065431</td>
      <td>906438</td>
      <td>7717402</td>
      <td>399656</td>
      <td>204113</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>81</td>
      <td>0.272814</td>
      <td>0.715717</td>
      <td>0.065829</td>
      <td>910421</td>
      <td>7709741</td>
      <td>407317</td>
      <td>200130</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>82</td>
      <td>0.273113</td>
      <td>0.715563</td>
      <td>0.066216</td>
      <td>914322</td>
      <td>7702271</td>
      <td>414787</td>
      <td>196229</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>83</td>
      <td>0.273216</td>
      <td>0.715361</td>
      <td>0.066618</td>
      <td>918142</td>
      <td>7694745</td>
      <td>422313</td>
      <td>192409</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>84</td>
      <td>0.273085</td>
      <td>0.715095</td>
      <td>0.067040</td>
      <td>921890</td>
      <td>7687104</td>
      <td>429954</td>
      <td>188661</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>85</td>
      <td>0.272377</td>
      <td>0.714650</td>
      <td>0.067504</td>
      <td>925386</td>
      <td>7679325</td>
      <td>437733</td>
      <td>185165</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>86</td>
      <td>0.270694</td>
      <td>0.713891</td>
      <td>0.068038</td>
      <td>928448</td>
      <td>7671333</td>
      <td>445725</td>
      <td>182103</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>87</td>
      <td>0.267922</td>
      <td>0.712777</td>
      <td>0.068655</td>
      <td>931066</td>
      <td>7663018</td>
      <td>454040</td>
      <td>179485</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>88</td>
      <td>0.264190</td>
      <td>0.711356</td>
      <td>0.069349</td>
      <td>933326</td>
      <td>7654353</td>
      <td>462705</td>
      <td>177225</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>89</td>
      <td>0.259901</td>
      <td>0.709763</td>
      <td>0.070112</td>
      <td>935597</td>
      <td>7645048</td>
      <td>472010</td>
      <td>174954</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>90</td>
      <td>0.255460</td>
      <td>0.708143</td>
      <td>0.070891</td>
      <td>937860</td>
      <td>7635590</td>
      <td>481468</td>
      <td>172691</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>91</td>
      <td>0.250997</td>
      <td>0.706543</td>
      <td>0.071676</td>
      <td>940143</td>
      <td>7626068</td>
      <td>490990</td>
      <td>170408</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>92</td>
      <td>0.246483</td>
      <td>0.704954</td>
      <td>0.072465</td>
      <td>942415</td>
      <td>7616511</td>
      <td>500547</td>
      <td>168136</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>93</td>
      <td>0.242030</td>
      <td>0.703412</td>
      <td>0.073249</td>
      <td>944700</td>
      <td>7606996</td>
      <td>510062</td>
      <td>165851</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>94</td>
      <td>0.237703</td>
      <td>0.701939</td>
      <td>0.074011</td>
      <td>946923</td>
      <td>7597744</td>
      <td>519314</td>
      <td>163628</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>95</td>
      <td>0.234467</td>
      <td>0.700856</td>
      <td>0.074642</td>
      <td>949156</td>
      <td>7589685</td>
      <td>527373</td>
      <td>161395</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>96</td>
      <td>0.232438</td>
      <td>0.700192</td>
      <td>0.075130</td>
      <td>951408</td>
      <td>7582927</td>
      <td>534131</td>
      <td>159143</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>97</td>
      <td>0.231602</td>
      <td>0.699935</td>
      <td>0.075477</td>
      <td>953678</td>
      <td>7577459</td>
      <td>539599</td>
      <td>156873</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>98</td>
      <td>0.231991</td>
      <td>0.700090</td>
      <td>0.075678</td>
      <td>955965</td>
      <td>7573317</td>
      <td>543741</td>
      <td>154586</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>99</td>
      <td>0.233285</td>
      <td>0.700546</td>
      <td>0.075773</td>
      <td>958275</td>
      <td>7570134</td>
      <td>546924</td>
      <td>152276</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>100</td>
      <td>0.234828</td>
      <td>0.701087</td>
      <td>0.075840</td>
      <td>960609</td>
      <td>7567180</td>
      <td>549878</td>
      <td>149942</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>101</td>
      <td>0.236613</td>
      <td>0.701708</td>
      <td>0.075882</td>
      <td>962985</td>
      <td>7564410</td>
      <td>552648</td>
      <td>147566</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>102</td>
      <td>0.238458</td>
      <td>0.702350</td>
      <td>0.075914</td>
      <td>965326</td>
      <td>7561777</td>
      <td>555281</td>
      <td>145225</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>103</td>
      <td>0.240473</td>
      <td>0.703048</td>
      <td>0.075927</td>
      <td>967685</td>
      <td>7559296</td>
      <td>557762</td>
      <td>142866</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>104</td>
      <td>0.242680</td>
      <td>0.703810</td>
      <td>0.075918</td>
      <td>970051</td>
      <td>7557016</td>
      <td>560042</td>
      <td>140500</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>105</td>
      <td>0.245113</td>
      <td>0.704645</td>
      <td>0.075883</td>
      <td>972427</td>
      <td>7554965</td>
      <td>562093</td>
      <td>138124</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>106</td>
      <td>0.247736</td>
      <td>0.705542</td>
      <td>0.075824</td>
      <td>974802</td>
      <td>7553129</td>
      <td>563929</td>
      <td>135749</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>107</td>
      <td>0.250555</td>
      <td>0.706502</td>
      <td>0.075742</td>
      <td>977176</td>
      <td>7551511</td>
      <td>565547</td>
      <td>133375</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>108</td>
      <td>0.253552</td>
      <td>0.707518</td>
      <td>0.075638</td>
      <td>979542</td>
      <td>7550107</td>
      <td>566951</td>
      <td>131009</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>109</td>
      <td>0.256716</td>
      <td>0.708588</td>
      <td>0.075513</td>
      <td>981896</td>
      <td>7548913</td>
      <td>568145</td>
      <td>128655</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>110</td>
      <td>0.260030</td>
      <td>0.709705</td>
      <td>0.075367</td>
      <td>984231</td>
      <td>7547924</td>
      <td>569134</td>
      <td>126320</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>111</td>
      <td>0.263468</td>
      <td>0.710861</td>
      <td>0.075205</td>
      <td>986553</td>
      <td>7547098</td>
      <td>569960</td>
      <td>123998</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>112</td>
      <td>0.266976</td>
      <td>0.712038</td>
      <td>0.075031</td>
      <td>988847</td>
      <td>7546406</td>
      <td>570652</td>
      <td>121704</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>113</td>
      <td>0.270445</td>
      <td>0.713200</td>
      <td>0.074857</td>
      <td>991092</td>
      <td>7545768</td>
      <td>571290</td>
      <td>119459</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>114</td>
      <td>0.273683</td>
      <td>0.714288</td>
      <td>0.074713</td>
      <td>993358</td>
      <td>7544832</td>
      <td>572226</td>
      <td>117193</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>115</td>
      <td>0.276283</td>
      <td>0.715170</td>
      <td>0.074641</td>
      <td>995586</td>
      <td>7543263</td>
      <td>573795</td>
      <td>114965</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>116</td>
      <td>0.278323</td>
      <td>0.715868</td>
      <td>0.074609</td>
      <td>997559</td>
      <td>7541583</td>
      <td>575475</td>
      <td>112992</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>117</td>
      <td>0.280142</td>
      <td>0.716493</td>
      <td>0.074593</td>
      <td>999428</td>
      <td>7539865</td>
      <td>577193</td>
      <td>111123</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>118</td>
      <td>0.281889</td>
      <td>0.717092</td>
      <td>0.074574</td>
      <td>1001194</td>
      <td>7538273</td>
      <td>578785</td>
      <td>109357</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>119</td>
      <td>0.283568</td>
      <td>0.717668</td>
      <td>0.074552</td>
      <td>1002858</td>
      <td>7536810</td>
      <td>580248</td>
      <td>107693</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>120</td>
      <td>0.285153</td>
      <td>0.718210</td>
      <td>0.074529</td>
      <td>1004405</td>
      <td>7535476</td>
      <td>581582</td>
      <td>106146</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>121</td>
      <td>0.286639</td>
      <td>0.718719</td>
      <td>0.074506</td>
      <td>1005837</td>
      <td>7534262</td>
      <td>582796</td>
      <td>104714</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>122</td>
      <td>0.288012</td>
      <td>0.719188</td>
      <td>0.074483</td>
      <td>1007148</td>
      <td>7533165</td>
      <td>583893</td>
      <td>103403</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>123</td>
      <td>0.289265</td>
      <td>0.719617</td>
      <td>0.074461</td>
      <td>1008338</td>
      <td>7532177</td>
      <td>584881</td>
      <td>102213</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>124</td>
      <td>0.290383</td>
      <td>0.719999</td>
      <td>0.074441</td>
      <td>1009396</td>
      <td>7531302</td>
      <td>585756</td>
      <td>101155</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>125</td>
      <td>0.291330</td>
      <td>0.720323</td>
      <td>0.074426</td>
      <td>1010307</td>
      <td>7530532</td>
      <td>586526</td>
      <td>100244</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>126</td>
      <td>0.292067</td>
      <td>0.720576</td>
      <td>0.074419</td>
      <td>1011068</td>
      <td>7529828</td>
      <td>587230</td>
      <td>99483</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>127</td>
      <td>0.292479</td>
      <td>0.720723</td>
      <td>0.074435</td>
      <td>1011666</td>
      <td>7529090</td>
      <td>587968</td>
      <td>98885</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>128</td>
      <td>0.292537</td>
      <td>0.720753</td>
      <td>0.074474</td>
      <td>1012095</td>
      <td>7528296</td>
      <td>588762</td>
      <td>98456</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>129</td>
      <td>-0.557920</td>
      <td>0.535480</td>
      <td>0.180431</td>
      <td>1045345</td>
      <td>6517320</td>
      <td>1599738</td>
      <td>65206</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>130</td>
      <td>-0.595328</td>
      <td>0.529343</td>
      <td>0.184933</td>
      <td>1045345</td>
      <td>6475777</td>
      <td>1641281</td>
      <td>65206</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>131</td>
      <td>-0.632858</td>
      <td>0.523304</td>
      <td>0.189450</td>
      <td>1045345</td>
      <td>6434098</td>
      <td>1682960</td>
      <td>65206</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>132</td>
      <td>-0.669836</td>
      <td>0.517467</td>
      <td>0.193900</td>
      <td>1045345</td>
      <td>6393032</td>
      <td>1724026</td>
      <td>65206</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>133</td>
      <td>-0.703920</td>
      <td>0.512523</td>
      <td>0.198122</td>
      <td>1046452</td>
      <td>6352966</td>
      <td>1764092</td>
      <td>64099</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>134</td>
      <td>-0.736712</td>
      <td>0.508089</td>
      <td>0.202264</td>
      <td>1048253</td>
      <td>6312947</td>
      <td>1804111</td>
      <td>62298</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>135</td>
      <td>-0.769210</td>
      <td>0.503828</td>
      <td>0.206383</td>
      <td>1050174</td>
      <td>6273014</td>
      <td>1844044</td>
      <td>60377</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>136</td>
      <td>-0.801483</td>
      <td>0.499719</td>
      <td>0.210485</td>
      <td>1052184</td>
      <td>6233153</td>
      <td>1883905</td>
      <td>58367</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>137</td>
      <td>-0.833567</td>
      <td>0.495744</td>
      <td>0.214571</td>
      <td>1054262</td>
      <td>6193366</td>
      <td>1923692</td>
      <td>56289</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>138</td>
      <td>-0.866197</td>
      <td>0.491664</td>
      <td>0.218687</td>
      <td>1056005</td>
      <td>6153643</td>
      <td>1963415</td>
      <td>54546</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>139</td>
      <td>-0.899758</td>
      <td>0.487423</td>
      <td>0.222879</td>
      <td>1057413</td>
      <td>6113556</td>
      <td>2003502</td>
      <td>53138</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>140</td>
      <td>-0.933823</td>
      <td>0.483167</td>
      <td>0.227123</td>
      <td>1058744</td>
      <td>6073063</td>
      <td>2043995</td>
      <td>51807</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>141</td>
      <td>-0.968472</td>
      <td>0.478830</td>
      <td>0.231412</td>
      <td>1059840</td>
      <td>6032391</td>
      <td>2084667</td>
      <td>50711</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>142</td>
      <td>-1.000000</td>
      <td>0.474437</td>
      <td>0.235744</td>
      <td>1060754</td>
      <td>5991501</td>
      <td>2125557</td>
      <td>49797</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>143</td>
      <td>-1.000000</td>
      <td>0.469924</td>
      <td>0.240125</td>
      <td>1061361</td>
      <td>5950465</td>
      <td>2166593</td>
      <td>49190</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>144</td>
      <td>-1.000000</td>
      <td>0.465222</td>
      <td>0.244559</td>
      <td>1061525</td>
      <td>5909393</td>
      <td>2207665</td>
      <td>49026</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>145</td>
      <td>-1.000000</td>
      <td>0.460654</td>
      <td>0.248961</td>
      <td>1061754</td>
      <td>5868543</td>
      <td>2248515</td>
      <td>48797</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>146</td>
      <td>-1.000000</td>
      <td>0.456673</td>
      <td>0.253201</td>
      <td>1062732</td>
      <td>5828441</td>
      <td>2288617</td>
      <td>47819</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>147</td>
      <td>-1.000000</td>
      <td>0.453373</td>
      <td>0.257226</td>
      <td>1064561</td>
      <td>5789466</td>
      <td>2327592</td>
      <td>45990</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>148</td>
      <td>-1.000000</td>
      <td>0.450564</td>
      <td>0.261168</td>
      <td>1067131</td>
      <td>5750523</td>
      <td>2366535</td>
      <td>43420</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>149</td>
      <td>-1.000000</td>
      <td>0.447731</td>
      <td>0.265118</td>
      <td>1069550</td>
      <td>5711658</td>
      <td>2405400</td>
      <td>41001</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>150</td>
      <td>-1.000000</td>
      <td>0.444799</td>
      <td>0.269033</td>
      <td>1071573</td>
      <td>5673507</td>
      <td>2443551</td>
      <td>38978</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>151</td>
      <td>-1.000000</td>
      <td>0.442356</td>
      <td>0.272797</td>
      <td>1074210</td>
      <td>5636138</td>
      <td>2480920</td>
      <td>36341</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>152</td>
      <td>-1.000000</td>
      <td>0.440458</td>
      <td>0.276353</td>
      <td>1077476</td>
      <td>5600053</td>
      <td>2517005</td>
      <td>33075</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>153</td>
      <td>-1.000000</td>
      <td>0.437252</td>
      <td>0.280286</td>
      <td>1078653</td>
      <td>5562590</td>
      <td>2554468</td>
      <td>31898</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>154</td>
      <td>-1.000000</td>
      <td>0.433695</td>
      <td>0.284440</td>
      <td>1079435</td>
      <td>5523474</td>
      <td>2593584</td>
      <td>31116</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>155</td>
      <td>-1.000000</td>
      <td>0.430392</td>
      <td>0.288531</td>
      <td>1080525</td>
      <td>5484635</td>
      <td>2632423</td>
      <td>30026</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>156</td>
      <td>-1.000000</td>
      <td>0.427311</td>
      <td>0.292588</td>
      <td>1081918</td>
      <td>5445805</td>
      <td>2671253</td>
      <td>28633</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>157</td>
      <td>-1.000000</td>
      <td>0.424204</td>
      <td>0.296704</td>
      <td>1083275</td>
      <td>5406461</td>
      <td>2710597</td>
      <td>27276</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>158</td>
      <td>-1.000000</td>
      <td>0.421240</td>
      <td>0.300842</td>
      <td>1084873</td>
      <td>5366686</td>
      <td>2750372</td>
      <td>25678</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>159</td>
      <td>-1.000000</td>
      <td>0.418245</td>
      <td>0.305018</td>
      <td>1086385</td>
      <td>5326633</td>
      <td>2790425</td>
      <td>24166</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>160</td>
      <td>-1.000000</td>
      <td>0.415058</td>
      <td>0.309264</td>
      <td>1087532</td>
      <td>5286314</td>
      <td>2830744</td>
      <td>23019</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>161</td>
      <td>-1.000000</td>
      <td>0.411385</td>
      <td>0.313655</td>
      <td>1087849</td>
      <td>5245477</td>
      <td>2871581</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>162</td>
      <td>-1.000000</td>
      <td>0.407478</td>
      <td>0.318190</td>
      <td>1087849</td>
      <td>5203629</td>
      <td>2913429</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>163</td>
      <td>-1.000000</td>
      <td>0.403591</td>
      <td>0.322754</td>
      <td>1087849</td>
      <td>5161509</td>
      <td>2955549</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>164</td>
      <td>-1.000000</td>
      <td>0.399747</td>
      <td>0.327319</td>
      <td>1087849</td>
      <td>5119389</td>
      <td>2997669</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>165</td>
      <td>-1.000000</td>
      <td>0.395945</td>
      <td>0.331883</td>
      <td>1087849</td>
      <td>5077269</td>
      <td>3039789</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>166</td>
      <td>-1.000000</td>
      <td>0.392185</td>
      <td>0.336448</td>
      <td>1087849</td>
      <td>5035149</td>
      <td>3081909</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>167</td>
      <td>-1.000000</td>
      <td>0.388464</td>
      <td>0.341013</td>
      <td>1087849</td>
      <td>4993029</td>
      <td>3124029</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>168</td>
      <td>-1.000000</td>
      <td>0.384783</td>
      <td>0.345577</td>
      <td>1087849</td>
      <td>4950909</td>
      <td>3166149</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>169</td>
      <td>-1.000000</td>
      <td>0.381139</td>
      <td>0.350142</td>
      <td>1087849</td>
      <td>4908789</td>
      <td>3208269</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>170</td>
      <td>-1.000000</td>
      <td>0.377531</td>
      <td>0.354706</td>
      <td>1087849</td>
      <td>4866669</td>
      <td>3250389</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>171</td>
      <td>-1.000000</td>
      <td>0.373960</td>
      <td>0.359271</td>
      <td>1087849</td>
      <td>4824549</td>
      <td>3292509</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>172</td>
      <td>-1.000000</td>
      <td>0.370422</td>
      <td>0.363835</td>
      <td>1087849</td>
      <td>4782429</td>
      <td>3334629</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>173</td>
      <td>-1.000000</td>
      <td>0.366919</td>
      <td>0.368400</td>
      <td>1087849</td>
      <td>4740309</td>
      <td>3376749</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>174</td>
      <td>-1.000000</td>
      <td>0.363448</td>
      <td>0.372965</td>
      <td>1087849</td>
      <td>4698189</td>
      <td>3418869</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>175</td>
      <td>-1.000000</td>
      <td>0.360009</td>
      <td>0.377529</td>
      <td>1087849</td>
      <td>4656069</td>
      <td>3460989</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>176</td>
      <td>-1.000000</td>
      <td>0.356601</td>
      <td>0.382094</td>
      <td>1087849</td>
      <td>4613949</td>
      <td>3503109</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>177</td>
      <td>-1.000000</td>
      <td>0.353223</td>
      <td>0.386658</td>
      <td>1087849</td>
      <td>4571829</td>
      <td>3545229</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>178</td>
      <td>-1.000000</td>
      <td>0.349874</td>
      <td>0.391223</td>
      <td>1087849</td>
      <td>4529709</td>
      <td>3587349</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>179</td>
      <td>-1.000000</td>
      <td>0.346554</td>
      <td>0.395787</td>
      <td>1087849</td>
      <td>4487589</td>
      <td>3629469</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>180</td>
      <td>-1.000000</td>
      <td>0.343261</td>
      <td>0.400352</td>
      <td>1087849</td>
      <td>4445469</td>
      <td>3671589</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>181</td>
      <td>-1.000000</td>
      <td>0.339995</td>
      <td>0.404916</td>
      <td>1087849</td>
      <td>4403349</td>
      <td>3713709</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>182</td>
      <td>-1.000000</td>
      <td>0.336754</td>
      <td>0.409481</td>
      <td>1087849</td>
      <td>4361229</td>
      <td>3755829</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>183</td>
      <td>-1.000000</td>
      <td>0.333540</td>
      <td>0.414046</td>
      <td>1087849</td>
      <td>4319109</td>
      <td>3797949</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>184</td>
      <td>-1.000000</td>
      <td>0.330349</td>
      <td>0.418610</td>
      <td>1087849</td>
      <td>4276989</td>
      <td>3840069</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>185</td>
      <td>-1.000000</td>
      <td>0.327183</td>
      <td>0.423175</td>
      <td>1087849</td>
      <td>4234869</td>
      <td>3882189</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>186</td>
      <td>-1.000000</td>
      <td>0.324039</td>
      <td>0.427739</td>
      <td>1087849</td>
      <td>4192749</td>
      <td>3924309</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>187</td>
      <td>-1.000000</td>
      <td>0.320918</td>
      <td>0.432304</td>
      <td>1087849</td>
      <td>4150629</td>
      <td>3966429</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>188</td>
      <td>-1.000000</td>
      <td>0.317819</td>
      <td>0.436868</td>
      <td>1087849</td>
      <td>4108509</td>
      <td>4008549</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>189</td>
      <td>-1.000000</td>
      <td>0.314741</td>
      <td>0.441433</td>
      <td>1087849</td>
      <td>4066389</td>
      <td>4050669</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>190</td>
      <td>-1.000000</td>
      <td>0.311683</td>
      <td>0.445998</td>
      <td>1087849</td>
      <td>4024269</td>
      <td>4092789</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>191</td>
      <td>-1.000000</td>
      <td>0.308646</td>
      <td>0.450562</td>
      <td>1087849</td>
      <td>3982149</td>
      <td>4134909</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>192</td>
      <td>-1.000000</td>
      <td>0.305627</td>
      <td>0.455127</td>
      <td>1087849</td>
      <td>3940029</td>
      <td>4177029</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>193</td>
      <td>-1.000000</td>
      <td>0.302627</td>
      <td>0.459691</td>
      <td>1087849</td>
      <td>3897909</td>
      <td>4219149</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>194</td>
      <td>-1.000000</td>
      <td>0.299645</td>
      <td>0.464256</td>
      <td>1087849</td>
      <td>3855789</td>
      <td>4261269</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>195</td>
      <td>-1.000000</td>
      <td>0.296681</td>
      <td>0.468820</td>
      <td>1087849</td>
      <td>3813669</td>
      <td>4303389</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>196</td>
      <td>-1.000000</td>
      <td>0.293733</td>
      <td>0.473385</td>
      <td>1087849</td>
      <td>3771549</td>
      <td>4345509</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>197</td>
      <td>-1.000000</td>
      <td>0.290922</td>
      <td>0.477762</td>
      <td>1087849</td>
      <td>3731162</td>
      <td>4385896</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>198</td>
      <td>-1.000000</td>
      <td>0.288296</td>
      <td>0.481871</td>
      <td>1087849</td>
      <td>3693247</td>
      <td>4423811</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>199</td>
      <td>-1.000000</td>
      <td>0.285710</td>
      <td>0.485936</td>
      <td>1087849</td>
      <td>3655737</td>
      <td>4461321</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>200</td>
      <td>-1.000000</td>
      <td>0.283158</td>
      <td>0.489966</td>
      <td>1087849</td>
      <td>3618542</td>
      <td>4498516</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>201</td>
      <td>-1.000000</td>
      <td>0.280650</td>
      <td>0.493943</td>
      <td>1087849</td>
      <td>3581849</td>
      <td>4535209</td>
      <td>22702</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>202</td>
      <td>-1.000000</td>
      <td>0.278251</td>
      <td>0.497836</td>
      <td>1087932</td>
      <td>3545843</td>
      <td>4571215</td>
      <td>22619</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>203</td>
      <td>-1.000000</td>
      <td>0.276348</td>
      <td>0.501526</td>
      <td>1088669</td>
      <td>3511050</td>
      <td>4606008</td>
      <td>21882</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>204</td>
      <td>-1.000000</td>
      <td>0.275096</td>
      <td>0.505036</td>
      <td>1090362</td>
      <td>3476971</td>
      <td>4640087</td>
      <td>20189</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>205</td>
      <td>-1.000000</td>
      <td>0.274557</td>
      <td>0.508263</td>
      <td>1093001</td>
      <td>3444554</td>
      <td>4672504</td>
      <td>17550</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>206</td>
      <td>-1.000000</td>
      <td>0.274249</td>
      <td>0.511258</td>
      <td>1095771</td>
      <td>3414146</td>
      <td>4702912</td>
      <td>14780</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>207</td>
      <td>-1.000000</td>
      <td>0.274069</td>
      <td>0.514101</td>
      <td>1098576</td>
      <td>3385113</td>
      <td>4731945</td>
      <td>11975</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>208</td>
      <td>-1.000000</td>
      <td>0.274288</td>
      <td>0.516743</td>
      <td>1101852</td>
      <td>3357452</td>
      <td>4759606</td>
      <td>8699</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>209</td>
      <td>-1.000000</td>
      <td>0.274549</td>
      <td>0.519351</td>
      <td>1105137</td>
      <td>3330105</td>
      <td>4786953</td>
      <td>5414</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>210</td>
      <td>-1.000000</td>
      <td>0.274498</td>
      <td>0.522056</td>
      <td>1107946</td>
      <td>3302336</td>
      <td>4814722</td>
      <td>2605</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>211</td>
      <td>-1.000000</td>
      <td>0.273837</td>
      <td>0.524780</td>
      <td>1109672</td>
      <td>3275474</td>
      <td>4841584</td>
      <td>879</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>212</td>
      <td>-1.000000</td>
      <td>0.272639</td>
      <td>0.527612</td>
      <td>1110550</td>
      <td>3248458</td>
      <td>4868600</td>
      <td>1</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>213</td>
      <td>-1.000000</td>
      <td>0.270893</td>
      <td>0.530538</td>
      <td>1110551</td>
      <td>3221459</td>
      <td>4895599</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>214</td>
      <td>-1.000000</td>
      <td>0.269210</td>
      <td>0.533366</td>
      <td>1110551</td>
      <td>3195365</td>
      <td>4921693</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>215</td>
      <td>-1.000000</td>
      <td>0.267552</td>
      <td>0.536157</td>
      <td>1110551</td>
      <td>3169607</td>
      <td>4947451</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>216</td>
      <td>-1.000000</td>
      <td>0.265876</td>
      <td>0.538986</td>
      <td>1110551</td>
      <td>3143509</td>
      <td>4973549</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>217</td>
      <td>-1.000000</td>
      <td>0.264215</td>
      <td>0.541795</td>
      <td>1110551</td>
      <td>3117584</td>
      <td>4999474</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>218</td>
      <td>-1.000000</td>
      <td>0.262558</td>
      <td>0.544604</td>
      <td>1110551</td>
      <td>3091667</td>
      <td>5025391</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>219</td>
      <td>-1.000000</td>
      <td>0.260905</td>
      <td>0.547413</td>
      <td>1110551</td>
      <td>3065745</td>
      <td>5051313</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>220</td>
      <td>-1.000000</td>
      <td>0.259236</td>
      <td>0.550253</td>
      <td>1110551</td>
      <td>3039536</td>
      <td>5077522</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>221</td>
      <td>-1.000000</td>
      <td>0.257530</td>
      <td>0.553163</td>
      <td>1110551</td>
      <td>3012683</td>
      <td>5104375</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>222</td>
      <td>-1.000000</td>
      <td>0.255796</td>
      <td>0.556129</td>
      <td>1110551</td>
      <td>2985318</td>
      <td>5131740</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>223</td>
      <td>-1.000000</td>
      <td>0.254009</td>
      <td>0.559189</td>
      <td>1110551</td>
      <td>2957079</td>
      <td>5159979</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>224</td>
      <td>-1.000000</td>
      <td>0.252156</td>
      <td>0.562369</td>
      <td>1110551</td>
      <td>2927734</td>
      <td>5189324</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>225</td>
      <td>-1.000000</td>
      <td>0.250251</td>
      <td>0.565646</td>
      <td>1110551</td>
      <td>2897500</td>
      <td>5219558</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>226</td>
      <td>-1.000000</td>
      <td>0.248306</td>
      <td>0.568997</td>
      <td>1110551</td>
      <td>2866577</td>
      <td>5250481</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>227</td>
      <td>-1.000000</td>
      <td>0.246265</td>
      <td>0.572522</td>
      <td>1110551</td>
      <td>2834051</td>
      <td>5283007</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>228</td>
      <td>-1.000000</td>
      <td>0.244054</td>
      <td>0.576346</td>
      <td>1110551</td>
      <td>2798761</td>
      <td>5318297</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>229</td>
      <td>-1.000000</td>
      <td>0.241676</td>
      <td>0.580469</td>
      <td>1110551</td>
      <td>2760720</td>
      <td>5356338</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>230</td>
      <td>-1.000000</td>
      <td>0.239193</td>
      <td>0.584781</td>
      <td>1110551</td>
      <td>2720929</td>
      <td>5396129</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>231</td>
      <td>-1.000000</td>
      <td>0.236601</td>
      <td>0.589292</td>
      <td>1110551</td>
      <td>2679300</td>
      <td>5437758</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>232</td>
      <td>-1.000000</td>
      <td>0.233982</td>
      <td>0.593857</td>
      <td>1110551</td>
      <td>2637180</td>
      <td>5479878</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>233</td>
      <td>-1.000000</td>
      <td>0.231368</td>
      <td>0.598421</td>
      <td>1110551</td>
      <td>2595060</td>
      <td>5521998</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>234</td>
      <td>-1.000000</td>
      <td>0.228758</td>
      <td>0.602986</td>
      <td>1110551</td>
      <td>2552940</td>
      <td>5564118</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>235</td>
      <td>-1.000000</td>
      <td>0.226150</td>
      <td>0.607550</td>
      <td>1110551</td>
      <td>2510820</td>
      <td>5606238</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>236</td>
      <td>-1.000000</td>
      <td>0.223545</td>
      <td>0.612115</td>
      <td>1110551</td>
      <td>2468700</td>
      <td>5648358</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>237</td>
      <td>-1.000000</td>
      <td>0.220943</td>
      <td>0.616680</td>
      <td>1110551</td>
      <td>2426580</td>
      <td>5690478</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>238</td>
      <td>-1.000000</td>
      <td>0.218342</td>
      <td>0.621244</td>
      <td>1110551</td>
      <td>2384460</td>
      <td>5732598</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>239</td>
      <td>-1.000000</td>
      <td>0.215742</td>
      <td>0.625809</td>
      <td>1110551</td>
      <td>2342340</td>
      <td>5774718</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>240</td>
      <td>-1.000000</td>
      <td>0.213142</td>
      <td>0.630373</td>
      <td>1110551</td>
      <td>2300220</td>
      <td>5816838</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>241</td>
      <td>-1.000000</td>
      <td>0.210543</td>
      <td>0.634938</td>
      <td>1110551</td>
      <td>2258100</td>
      <td>5858958</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>242</td>
      <td>-1.000000</td>
      <td>0.207943</td>
      <td>0.639502</td>
      <td>1110551</td>
      <td>2215980</td>
      <td>5901078</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>243</td>
      <td>-1.000000</td>
      <td>0.205341</td>
      <td>0.644067</td>
      <td>1110551</td>
      <td>2173860</td>
      <td>5943198</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>244</td>
      <td>-1.000000</td>
      <td>0.202738</td>
      <td>0.648632</td>
      <td>1110551</td>
      <td>2131740</td>
      <td>5985318</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>245</td>
      <td>-1.000000</td>
      <td>0.200132</td>
      <td>0.653196</td>
      <td>1110551</td>
      <td>2089620</td>
      <td>6027438</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>246</td>
      <td>-1.000000</td>
      <td>0.197522</td>
      <td>0.657761</td>
      <td>1110551</td>
      <td>2047500</td>
      <td>6069558</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>247</td>
      <td>-1.000000</td>
      <td>0.194909</td>
      <td>0.662325</td>
      <td>1110551</td>
      <td>2005380</td>
      <td>6111678</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>248</td>
      <td>-1.000000</td>
      <td>0.192292</td>
      <td>0.666890</td>
      <td>1110551</td>
      <td>1963260</td>
      <td>6153798</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>249</td>
      <td>-1.000000</td>
      <td>0.189669</td>
      <td>0.671454</td>
      <td>1110551</td>
      <td>1921140</td>
      <td>6195918</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>250</td>
      <td>-1.000000</td>
      <td>0.187040</td>
      <td>0.676019</td>
      <td>1110551</td>
      <td>1879020</td>
      <td>6238038</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>251</td>
      <td>-1.000000</td>
      <td>0.184404</td>
      <td>0.680583</td>
      <td>1110551</td>
      <td>1836900</td>
      <td>6280158</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>252</td>
      <td>-1.000000</td>
      <td>0.181760</td>
      <td>0.685148</td>
      <td>1110551</td>
      <td>1794780</td>
      <td>6322278</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>253</td>
      <td>-1.000000</td>
      <td>0.179108</td>
      <td>0.689713</td>
      <td>1110551</td>
      <td>1752660</td>
      <td>6364398</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
    <tr>
      <td>254</td>
      <td>-1.000000</td>
      <td>0.176446</td>
      <td>0.694277</td>
      <td>1110551</td>
      <td>1710540</td>
      <td>6406518</td>
      <td>0</td>
      <td>9573764</td>
      <td>9386189</td>
      <td>9227609</td>
    </tr>
  </tbody>
</table>

## Disclaimer

This software was developed at the National Institute of Standards
and Technology (NIST) by employees of the Federal Government in the
course of their official duties. Pursuant to Title 17 Section 105
of the United States Code, this software is not subject to copyright
protection and is in the public domain. NIST assumes no responsibility
whatsoever for use by other parties of its source code or open source
server, and makes no guarantees, expressed or implied, about its quality,
reliability, or any other characteristic.